<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-setup" data-toc-modified-id="Notebook-setup-1">Notebook setup</a></span></li><li><span><a href="#State/EPA-Residiental-Survey-(SRRS)-datasets" data-toc-modified-id="State/EPA-Residiental-Survey-(SRRS)-datasets-2">State/EPA Residiental Survey (SRRS) datasets</a></span></li></ul></div>

### Notebook setup

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

In [ ]:
# keep notebook outputs clean - demos only
import logging
logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)

**Best Practice:  avoid meaningless precision**

The precision of an estimate is inversely proportional to square root of the amount of data.
For the Minnesota data, only 2 decimal places is warrented.
To change the default print behavoir for pandas DataFrames, we use the pandas global option [display.precision](https://pandas.pydata.org/docs/user_guide/options.html#frequently-used-options).

In [ ]:
pd.set_option('display.precision', 2)

### State/EPA Residiental Survey (SRRS) datasets

The rawest form of the radon data was collected and archived by Phil Price and is available here:
http://www.stat.columbia.edu/~gelman/arm/examples/radon_complete


* The documentation is in file http://www.stat.columbia.edu/~gelman/arm/examples/radon_complete/SRRSdoc.pdf

* There are 5 files, srrs1.dat through srrs5.dat  - but data is duplicated between them.

* This directory also contains data from both national survey - NRRS - and state surveys - cf. https://link.springer.com/article/10.1007/BF02034901.   This is in a different format and is
not used in Gelman and Hill analysis.

* README notes that files are old backups, things may be missing.

Indian reservations vs. stolen lands:
the state/EPA survey contains data from Indian lands.  column 'STATE' code R5, R6, R7, RB, RC, RN.
EPA regions cross state boundaries; Indian lands cross county boundaries, e.g.
EPA region 5 covers Indian lands in MN, WI, and MI: https://www.epa.gov/sites/default/files/2015-08/documents/r5-tribal-land-map.pdf

Given this, we have two files:

* 59,396 radon_us.csv 
* 3,897 radon_indg.csv

If floor is unknown, coded as '9'

## Align, combine, subset EPA radon, uranium data and US Census GIS data

In [ ]:
us_radon = pd.read_csv(os.path.join('data','radon_us.csv'),
                     usecols=['state', 'stfips', 'floor', 'activity', 'county', 'cntyfips'],
                     skipinitialspace=True,    # CSV file has spaces after delimiter, ignore them
    ).convert_dtypes()
print(us_radon.shape)
us_radon[:3]

**datacleanup**

Colorado and CT have data with cntyfips codes '0' and '999'.   Dropping for now.

In [ ]:
us_radon.drop(us_radon[us_radon.cntyfips==0].index, inplace=True)
us_radon.drop(us_radon[us_radon.cntyfips==999].index, inplace=True)
print(us_radon.shape)

### US county soil uranium levels

Also distributed from Gelman website.

In [ ]:
us_uranium = pd.read_csv(os.path.join('data','raw_uranium.csv'),
                        usecols=['stfips', 'ctfips', 'st', 'cty', 'Uppm'],
                        skipinitialspace=True,
                        ).drop_duplicates().convert_dtypes()
us_uranium.head(3)

### US census county boundaries GIS files

In [ ]:
# gis data
import geopandas as gpd
import libpysal as sa

In [ ]:
shpfile = os.path.join('geo_data','cb_2018_us_county_20m', 'cb_2018_us_county_20m.shp')
us_geodata = gpd.read_file(shpfile)
print(type(us_geodata))
us_geodata.head(3)

### Join and merge tables using US FIPS codes

To join or merge tables, we need to create a common key in both, then
use the [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.


[FIPS code](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) are numbers which uniquely identify geographic areas. 
The US census datasets have "GEOID" code, the first 2 digits of which are the state FIPS code, the last 3 are the county-level FIPS code.
The other datasets have separate columns for stats and county codes.

In [ ]:
# drop data on ALASKA ('02'), HAWAII ('15'), PR ('72'), USVI ('78')
islands = ['02', '15', '72', '78']
us_geodata = us_geodata[~us_geodata.STATEFP.isin(islands)]
# statefips are ints in other tables
islands = [2, 15, 72, 78]
us_radon = us_radon[~us_radon.stfips.isin(islands)]
us_uranium = us_uranium[~us_uranium.stfips.isin(islands)]

In [ ]:
# create merge column
us_uranium['FIPS'] = us_uranium.stfips*1000 + us_uranium.ctfips
us_radon['FIPS'] = us_radon.stfips*1000 + us_radon.cntyfips

#### Add soil uranium 'Uppm' to GeoPandasDataFrame

In [ ]:
# GEOID should be numeric
us_geodata = us_geodata.astype({'GEOID': 'int32'}, copy=False)

# left join because Uranium dataset is missing one record
us_geodata = us_geodata.merge(us_uranium, how='left', left_on='GEOID', right_on='FIPS')

# cleanup missing value so that we can use column as fill value
us_geodata = us_geodata.astype({'Uppm': 'float64'})
us_geodata.Uppm.fillna(value=0, inplace=True)

# drop columns for cleaner display
us_geodata.drop(columns=['STATEFP','COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'LSAD', 'ALAND', 'AWATER',
                       'stfips', 'ctfips', 'cty', 'FIPS'], inplace=True)
us_geodata.rename(columns={'st':'STATE','NAME': 'county'}, inplace=True)

In [ ]:
us_geodata.head(3)

#### Add soil uranium to radon data

In [ ]:
us_radon.head(3)

In [ ]:
us_radon = us_radon.merge(us_geodata[['GEOID', 'Uppm', 'county']], left_on='FIPS', right_on='GEOID')
us_radon.head(3)

**Cleanup**

Remove the columns which contain redundant information.

In [ ]:
us_radon.drop(columns=['stfips', 'cntyfips', 'county_x', 'FIPS'], inplace=True)
us_radon.rename(columns={'county_y':'county'}, inplace=True)
us_radon.head(3)

**Restrict dataset to Minnesota**

In order to work with just the data from Minnesota, we use a 
use a conditional expression to [filter specific rows of a dataframe](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-filter-specific-rows-from-a-dataframe), combined with operation [reset_index(drop=True)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html?highlight=reset_index#pandas.DataFrame.reset_index) so that the rows are indexed starting from 0.

In [ ]:
mn_radon = us_radon[us_radon['state']=='MN'].reset_index(drop=True)
mn_radon.drop(columns=['state'], inplace=True)
mn_radon.head(3)

**Put data on log scale**

Following Gelman and Hill chapter 4, section 4, we work with data on the log scale,
for two reasons

+ the outcome variable log_radon is always positive.
+ it provides modeling flexibility.

We know from geology that both radon measurements and soil uranium levels are always greater than zero,
however a few radon measurements in the EPA dataset are 0.
In order to be able to work with these measurements on the log scale, we replace 0 with 0.1,
which corresponds to a low radon level (following Gelman and Hill).

In [ ]:
mn_radon['radon'] = mn_radon.activity.apply(lambda x: x if x > 0. else 0.1)
mn_radon['log_radon'] = np.log(mn_radon['radon'])

mn_radon['uranium'] = mn_radon.Uppm.apply(lambda x: x if x > 0. else 0.1)
mn_radon['log_uranium'] = np.log(mn_radon['uranium'])

mn_radon.drop(columns=['activity'], inplace=True)
mn_radon.head(3)

#### Unique county ids based on *all* counties in MN

(we know dataset is missing 2 counties - possibly indian reservations?)

In [ ]:
us_geodata.head(3)

In [ ]:
mn_geodata = us_geodata[us_geodata.STATE == 'MN'].reset_index(drop=True).reset_index()
mn_geodata['index'] = mn_geodata['index'] + 1
mn_geodata.rename(columns={'index':'county_id'}, inplace=True)
mn_geodata.head(3)

**Create auxiliary dataset of per-county information**

County-level information includes the number of observations taken in that county as well as the soil uranium level.
In order to easily visualize this information using plotnine, we create a secondary pandas.Dataframe object with per-county level information.

The [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html) method returns a Series containing counts of unique values,
We add these to the county-level dataframe.

In [ ]:
mn_radon = mn_radon.merge(mn_geodata[['county', 'county_id']], on='county')
homes = mn_radon.value_counts(subset='county', sort=False).to_frame(name='homes').reset_index()
mn_geodata = mn_geodata.merge(homes, how='left', on='county')
mn_geodata.homes.fillna(0, inplace=True)


**Save as CSV files**

These files are already part of this notebook, therefore calls to the  [pandas.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv) method have been commented out.

In [ ]:
mn_geodata.head(3)

In [ ]:
mn_radon.head(3)

In [ ]:
# uncomment as needed
mn_radon.to_csv(os.path.join('data', 'mn_radon.csv'), index=False)
mn_geodata.to_csv(os.path.join('data', 'mn_geodata.csv'), index=False)